In [1]:
import roboschool
import gym
import numpy as np
import math
from scipy import linalg
from numpy.linalg import matrix_rank
import time

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython 
import pickle

from scipy import linalg
from reachersim import ReacherSim

In [2]:
class PI_Blocks:
    def __init__(self):
        #here N is the number of disretized trajectory points
        self.N = 100
        #here K is the number of trajectories
        self.K = 5
        #J is the number of joints
        self.J = 5
        self.P = self.N*self.J
        self.iters = 1
        self.R = self.get_R()
        self.R_inv = linalg.inv(self.R)
        self.P = self.N*self.J
        self.A2 = np.zeros([self.P+2,self.P])
        self.R_j = self.get_R_j()
        self.R_j_inv = linalg.inv(self.R_j)
        self.th = np.zeros([self.P])
        self.noisy_trajs = np.zeros([self.K,self.P])
        self.M_arr = self.get_M()
        
        self.cost_control = np.zeros([self.K, self.N])
        self.cost_state = np.zeros([self.K, self.N])
        self.pos = np.zeros([2,self.K, self.N]) 
        
    def cost_inst(self,k,t):
        e = self.noisy_trajs[k] - self.th
        self.cost_control[k,t] = (self.th + self.M_arr[t]@e)@self.R_j@(self.th + self.M_arr[t]@e).T
        self.cost_state[k,t] = (self.get_acc(0,k,t))**2 + (self.get_acc(1,k,t))**2
        
    def get_R(self):
# #     first order finite differences
#         A1 = np.zeros([self.N+1,self.N])
#         for i in range (self.N+1):
#             for j in range(self.N):
#                 if(i==j):
#                     A1[i,j] = 1
#                     if(i>0 & i<self.N-1):
#                         A1[i,j-1] =-1
#         A1[self.N,self.N-1] = 1
#         return A1.T@A1

#     second order finite differences
        A2 = np.zeros([self.N+2,self.N])
        for i in range (self.N+2):
            for j in range(self.N):
                if(i==j):
                    A2[i,j] = 1
                    if(i>0 & i<self.N+1):
                        A2[i,j-1] =-2
                        if(i>1 & i<self.N+1):
                            A2[i,j-2] =1
        A2[self.N,self.N-2] = 1
        A2[self.N,self.N-1] = -2
        A2[self.N+1,self.N-1] = 1
        wd = 1.
        self.A2 = A2
        return wd*A2.T@A2
    
    
    def get_R_j(self):
        J = self.J
        N = self.N
        R = self.get_R()
#         print(R)
        R_j = np.zeros([J*N,J*N])
        for j in range(J):
            R_j[j*N:j*N+N,j*N:j*N+N] = R
        return R_j
        
    #creates an array of M matrices for each time step
    def get_M(self):
        M_arr = [] 
        J = self.J
        N = self.N
        for i in range(N):
            gt = np.zeros([J*N,J])
            for j in range(J):
                gt[j*N+i,j] = 1.
            den = (gt.T@(self.R_j_inv)@gt)
            M = (self.R_j_inv@gt@gt.T)/den[0,0]
            M_arr.append(M)
        return M_arr
    
    def init_traj(self,start,goal):
        J = self.J
        N = self.N
        for j in range(J):
            self.th[j*N:j*N+N] = np.linspace(start[j],goal[j],N)
        
    def create_noisy_traj(self):
        N = self.N
        mean = np.zeros([N])
        noise_scale = 0.01
        for k in range(self.K):
            for j in range(self.J):
                self.noisy_trajs[k,j*N:j*N+N] = self.th[j*N:j*N+N] + noise_scale*np.random.multivariate_normal(mean,self.R_inv)
#         print(self.noisy_trajs) 
#         plt.plot(range(self.N*self.J), self.noisy_trajs.T)
# #         plt.plot(range(self.N), self.R_inv)
#         plt.xlabel("time")
#         plt.ylabel("theta")
#         plt.show() 

    def get_acc(self,k,t):
        # pos is the matrix of positions at all time steps for a particular traj
        acc = self.A2[t+1,:]@self.pos[:,k,:].T
        return acc
    
    
    

In [3]:
class PathIntegral:
    
    def __init__(self, model, pi_blocks):
        
        #N is the number of disretized trajectory points
        self.N = 4
        #K is the number of trajectories
        self.K = 10
        #J is the number of joints
        self.J = 3
        self.pi_blocks = pi_blocks
        #No of parameter updates
        self.iters = 1
        self.sim_t = 800
        
        #setup environment & it checks if the target is reachable for the manipulator or not
        self.model = model
        self.state,self.a,self.env = self.model.setup_environment()
        #inverse kinematics of the target position, to get the desired state space of the reacher
        self.th1_target, self.th2_target = self.model.inverse_kinematics(self.state[0], self.state[1])
        
        
    def wrap_to_pi(self, th):
        th = (((th+np.pi) % (2*np.pi)) - np.pi)
        return th
    
    def rollout(self, k, th_params):
        t = 0
        for i in range(self.sim_t):
            t = int(np.clip(t, 0, self.N-1))
            
            #Reading current states
            th1 = self.wrap_to_pi(np.arctan2(self.state[5],self.state[4]))
            th2= self.wrap_to_pi((self.state[7]*3.14))
            
            th1_dot = self.state[6]
            th2_dot = self.state[8]
            
            self.pi_blocks.pos[:,k,t] = self.end_eff_pos(th1,th2)
            self.pi_blocks.cost_inst(k,t)
            
            if(t<self.N-1):
                
                self.a[0] = 2.5*(th_params[t] - th1) - 2*(th1_dot - 0)
                self.a[1] = 2.5*(th_params[t+self.N] - th2) - 2*(th2_dot - 0)
                
                self.state, _, _, _ = self.env.step(self.a)
                
            if(i%40 == 0):
                t = t+1
        
    def iterations(self,itr):
        
        for k in range(self.K):
            self.state, self.a, self.env = self.model.setup_environment()
            self.rollout(itr,k)
            
    
    def end_eff_pos(self,th1,th2):
        l1 = 0.1
        l2 = 0.11
        x = l1*np.cos(th1) +l2*np.cos(th1+th2)
        y = l1*np.sin(th1) +l2*np.sin(th1+th2)
        return x,y
    
    
    
    
        
    
    
    
        

In [4]:
model = ReacherSim()
param_obj = PI_Blocks()
PI = PathIntegral(model, param_obj)
for itr in range(PI.iters):
    PI.iterations(itr)
    

TypeError: get_acc() takes 3 positional arguments but 4 were given